In [32]:
!pip install -q transformers datasets peft bitsandbytes accelerate evaluate trl
!pip install -q sentencepiece protobuf
!pip install -q torch torchvision torchaudio

# LoRA Fine-Tuning

In [33]:
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
import time
import psutil
import os
from typing import Dict, List

In [34]:
def prepare_dataset(dataset_name: str = "imdb", max_samples: int = 5000):
    """
    Prepare dataset for fine-tuning. Using IMDB for sentiment analysis as example.
    """
    print(f"\n{'='*60}")
    print(f"Loading dataset: {dataset_name}")
    print(f"{'='*60}")

    if dataset_name == "imdb":
        dataset = load_dataset("imdb")
        # Reduce dataset size for faster training
        train_dataset = dataset["train"].shuffle(seed=42).select(range(max_samples))
        test_dataset = dataset["test"].shuffle(seed=42).select(range(500))

        # Format as instruction-following task
        def format_instruction(example):
            text = f"Classify the sentiment of this movie review as positive or negative.\n\nReview: {example['text'][:500]}\n\nSentiment:"
            label = " positive" if example["label"] == 1 else " negative"
            return {"text": text + label}

        train_dataset = train_dataset.map(format_instruction)
        test_dataset = test_dataset.map(format_instruction)

    return train_dataset, test_dataset

In [35]:
train_data, test_data = prepare_dataset("imdb", max_samples=3000)
print(f"Training samples: {len(train_data)}")
print(f"Test samples: {len(test_data)}")
print(f"\nExample training sample:\n{train_data[0]['text'][:300]}...")


Loading dataset: imdb
Training samples: 3000
Test samples: 500

Example training sample:
Classify the sentiment of this movie review as positive or negative.

Review: There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier's plot are far ...


In [36]:
MODEL_NAME = "facebook/opt-350m"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

def tokenize_function(examples):
    """Tokenize the text data"""
    outputs = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    outputs["labels"] = outputs["input_ids"].copy()
    return outputs

In [37]:
tokenized_train = train_data.map(
    tokenize_function,
    batched=True,
    remove_columns=train_data.column_names
)
tokenized_test = test_data.map(
    tokenize_function,
    batched=True,
    remove_columns=test_data.column_names
)

In [38]:
def get_model_size(model):
    """Calculate model size in MB"""
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    size_mb = (param_size + buffer_size) / 1024**2
    return size_mb

def count_parameters(model):
    """Count total and trainable parameters"""
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

def print_model_stats(model, method_name: str):
    """Print comprehensive model statistics"""
    total, trainable = count_parameters(model)
    print(f"\n{'='*60}")
    print(f"{method_name} - Model Statistics")
    print(f"{'='*60}")
    print(f"Total parameters: {total:,}")
    print(f"Trainable parameters: {trainable:,}")
    print(f"Trainable %: {100 * trainable / total:.2f}%")
    print(f"Model size: {get_model_size(model):.2f} MB")
    if torch.cuda.is_available():
        print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
        print(f"GPU memory cached: {torch.cuda.memory_reserved() / 1e9:.2f} GB")


In [39]:
BASE_TRAINING_ARGS = {
    "num_train_epochs": 1,
    "per_device_train_batch_size": 4,
    "per_device_eval_batch_size": 4,
    "gradient_accumulation_steps": 4,
    "learning_rate": 2e-4,
    "weight_decay": 0.01,
    "logging_steps": 50,
    "save_strategy": "no",
    "eval_strategy": "steps",
    "eval_steps": 200,
    "fp16": True,
    "report_to": "none",
}

In [40]:
# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Load base model
base_model_lora = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

In [41]:
lora_config = LoraConfig(
    r=16,  # Rank of the low-rank matrices
    lora_alpha=32,  # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Which layers to apply LoRA to
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

In [42]:
lora_model = get_peft_model(base_model_lora, lora_config)
print_model_stats(lora_model, "LoRA")

# Training arguments for LoRA
lora_training_args = TrainingArguments(
    **BASE_TRAINING_ARGS,
    output_dir="./results_lora",
)

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Create trainer
lora_trainer = Trainer(
    model=lora_model,
    args=lora_training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
)

The model is already on multiple devices. Skipping the move to device specified in `args`.



LoRA - Model Statistics
Total parameters: 332,769,280
Trainable parameters: 1,572,864
Trainable %: 0.47%
Model size: 637.71 MB
GPU memory allocated: 0.98 GB
GPU memory cached: 1.67 GB


In [43]:
# Train and measure time
print("Starting LoRA training...")
lora_start_time = time.time()
lora_results = lora_trainer.train()
lora_training_time = time.time() - lora_start_time

Starting LoRA training...


Step,Training Loss,Validation Loss


In [44]:
print(f"LoRA Training completed in {lora_training_time:.2f} seconds")
print(f"Final training loss: {lora_results.training_loss:.4f}")

LoRA Training completed in 238.64 seconds
Final training loss: 3.0122


In [45]:
lora_eval_results = lora_trainer.evaluate()
print(f"LoRA Evaluation loss: {lora_eval_results['eval_loss']:.4f}")

LoRA Evaluation loss: 2.8743


# QLoRA Fine-Tuning

In [46]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [47]:
# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [48]:
base_model_qlora = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

In [49]:
base_model_qlora = prepare_model_for_kbit_training(base_model_qlora)

qlora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

In [50]:
qlora_model = get_peft_model(base_model_qlora, qlora_config)
print_model_stats(qlora_model, "QLoRA")

qlora_training_args = TrainingArguments(
    **BASE_TRAINING_ARGS,
    output_dir="./results_qlora",
)


QLoRA - Model Statistics
Total parameters: 181,250,048
Trainable parameters: 1,572,864
Trainable %: 0.87%
Model size: 257.91 MB
GPU memory allocated: 1.27 GB
GPU memory cached: 1.73 GB


In [51]:
qlora_trainer = Trainer(
    model=qlora_model,
    args=qlora_training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
)

In [52]:
print("Starting QLoRA training...")
qlora_start_time = time.time()
qlora_results = qlora_trainer.train()
qlora_training_time = time.time() - qlora_start_time

Starting QLoRA training...


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss


In [53]:
print(f"\nQLoRA Training completed in {qlora_training_time:.2f} seconds")
print(f"Final training loss: {qlora_results.training_loss:.4f}")


QLoRA Training completed in 357.27 seconds
Final training loss: 3.0609


In [54]:
qlora_eval_results = qlora_trainer.evaluate()
print(f"QLoRA Evaluation loss: {qlora_eval_results['eval_loss']:.4f}")

QLoRA Evaluation loss: 2.9067


# Comprehensive Comparison

In [55]:
lora_total, lora_trainable = count_parameters(lora_model)
qlora_total, qlora_trainable = count_parameters(qlora_model)

comparison_data = {
    "Method": ["LoRA", "QLoRA"],
    "Total Params": [f"{lora_total:,}", f"{qlora_total:,}"],
    "Trainable Params": [f"{lora_trainable:,}", f"{qlora_trainable:,}"],
    "Trainable %": [
        f"{100 * lora_trainable / lora_total:.2f}%",
        f"{100 * qlora_trainable / qlora_total:.2f}%"
    ],
    "Training Time": [
        f"{lora_training_time:.2f}s",
        f"{qlora_training_time:.2f}s"
    ],
    "Final Train Loss": [
        f"{lora_results.training_loss:.4f}",
        f"{qlora_results.training_loss:.4f}"
    ],
    "Eval Loss": [
        f"{lora_eval_results['eval_loss']:.4f}",
        f"{qlora_eval_results['eval_loss']:.4f}"
    ],
    "Model Size": [
        f"{get_model_size(lora_model):.2f} MB",
        f"{get_model_size(qlora_model):.2f} MB"
    ]
}

In [56]:
for key in comparison_data:
    print(f"{key:20s} | {comparison_data[key][0]:30s} | {comparison_data[key][1]:30s}")
print("="*100)

# Key insights
print("\n" + "="*60)
print("KEY INSIGHTS")
print("="*60)
print(f"1. Memory Efficiency: QLoRA uses ~{get_model_size(lora_model) / get_model_size(qlora_model):.2f}x less memory than LoRA")
print(f"2. Training Speed: QLoRA is {lora_training_time / qlora_training_time:.2f}x {'faster' if qlora_training_time < lora_training_time else 'slower'} than LoRA")
print(f"3. Parameter Efficiency: Both methods train only {100 * lora_trainable / lora_total:.2f}% of parameters")
print(f"4. Performance: Evaluation loss difference: {abs(lora_eval_results['eval_loss'] - qlora_eval_results['eval_loss']):.4f}")

Method               | LoRA                           | QLoRA                         
Total Params         | 332,769,280                    | 181,250,048                   
Trainable Params     | 1,572,864                      | 1,572,864                     
Trainable %          | 0.47%                          | 0.87%                         
Training Time        | 238.64s                        | 357.27s                       
Final Train Loss     | 3.0122                         | 3.0609                        
Eval Loss            | 2.8743                         | 2.9067                        
Model Size           | 637.71 MB                      | 257.87 MB                     

KEY INSIGHTS
1. Memory Efficiency: QLoRA uses ~2.47x less memory than LoRA
2. Training Speed: QLoRA is 0.67x slower than LoRA
3. Parameter Efficiency: Both methods train only 0.47% of parameters
4. Performance: Evaluation loss difference: 0.0323


In [72]:
def generate_response(model, prompt: str, max_new_tokens: int = 15):
    """Generate text from a prompt"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_return_sequences=1,
            temperature=0.1,  # Very low temperature for deterministic output
            do_sample=False,  # Greedy decoding for most likely answer
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    generated_ids = outputs[0][inputs['input_ids'].shape[1]:]
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

    # Extract only the first word/line (the actual sentiment prediction)
    # Stop at newline or after first complete word
    if '\n' in generated_text:
        generated_text = generated_text.split('\n')[0].strip()

    # Take only the first word if it's a simple sentiment
    words = generated_text.split()
    if len(words) > 0 and words[0].lower() in ['positive', 'negative']:
        return words[0]

    return generated_text

In [73]:
test_prompts = [
    "Classify the sentiment of this movie review as positive or negative.\n\nReview: This film was absolutely incredible! The acting was superb.\n\nSentiment:",
    "Classify the sentiment of this movie review as positive or negative.\n\nReview: Terrible waste of time. I want my money back.\n\nSentiment:",
]

print("LoRA Model Predictions:")
print("-" * 60)
for i, prompt in enumerate(test_prompts, 1):
    response = generate_response(lora_model, prompt)
    print(f"Example {i}:")
    print(f"Prompt: {prompt[:100]}...")
    print(f"Predicted Sentiment: {response}")

print("QLoRA Model Predictions:")
print("-" * 60)
for i, prompt in enumerate(test_prompts, 1):
    response = generate_response(qlora_model, prompt)
    print(f"Example {i}:")
    print(f"Prompt: {prompt[:100]}...")
    print(f"Predicted Sentiment: {response}")


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


LoRA Model Predictions:
------------------------------------------------------------
Example 1:
Prompt: Classify the sentiment of this movie review as positive or negative.

Review: This film was absolute...
Predicted Sentiment: positive
Example 2:
Prompt: Classify the sentiment of this movie review as positive or negative.

Review: Terrible waste of time...
Predicted Sentiment: negative
QLoRA Model Predictions:
------------------------------------------------------------
Example 1:
Prompt: Classify the sentiment of this movie review as positive or negative.

Review: This film was absolute...
Predicted Sentiment: positive
Example 2:
Prompt: Classify the sentiment of this movie review as positive or negative.

Review: Terrible waste of time...
Predicted Sentiment: negative
